In [2]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [3]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, JLD

In [4]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )

@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end

TM (generic function with 1 method)

In [5]:
t = 200
tt = 500
tstep = 0.001
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [6]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  const U0 = 0.35;  const ΔU0 = 0.305;

In [7]:
I0 = -1.0
p =  [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [3.740482362235197, 0.8176653843010522, 0.4087238640418287]
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.35, 0.305, 0.3, -1.0]
 time:          0.0
 state:         [3.740482362235197, 0.8176653843010522, 0.4087238640418287]


In [8]:
I0range = range( -1.0, -2.0, length = 3000 )

idx_control_parameter = 11

idx_fix = 2; fixed_value = 0.725
surface = (idx_fix, fixed_value)
setting_root = (xrtol = 1e-15, atol = 1e-20);

In [9]:
pmap_ = PoincareMap(ds, surface, rootkw = setting_root)

3-dimensional PoincareMap
 deterministic: true
 discrete time: true
 in-place:      false
 dynamic rule:  TM
 hyperplane:    (2, 0.725)
 crossing time: 0.2981054443466382
 parameters:    [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.35, 0.305, 0.3, -1.0]
 time:          0
 state:         [6.57884585796208, 0.725, 0.3989831947860078]


In [10]:
idx_save = 1
output = orbitdiagram(pmap_, idx_save, idx_control_parameter, I0range; n = 1000, Ttr = 250);

In [ ]:
f = Figure(resolution = (1200, 600))
axis = Axis(f[1,1]; xlabel = L"I_0", ylabel = L"E", xlabelsize = 35, ylabelsize = 35,
            xticklabelsize = 30, yticklabelsize = 30, xgridvisible = false, ygridvisible = false)
for (j, p) in enumerate(I0range)
    scatter!(axis, fill(p, length(output[j])), output[j]; color = ("black", 0.5), markersize = 2.5, marker = :circle)
end
f

In [ ]:
save("BF_U0=0.35_extended.jld", "data", output)